LOAD DATA INTO PICKLE

In [3]:
import tensorflow as tf
import numpy as np
import os
import cv2
import random
import pickle

DIRECTORY = "C:/Users/Leon Chen/Documents/Projects/ML-basics/horse-or-human"
SETS = ["train", "validation"]
LABELS = ["horses", "humans"]
IMAGE_SIZE = 50

train_set = []
val_set = []

for set in SETS:
    for label in LABELS:
        path = os.path.join(DIRECTORY, set)
        path = os.path.join(path, label)
        label = LABELS.index(label)
        for img in os.listdir(path):
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.resize(img_arr, (IMAGE_SIZE, IMAGE_SIZE))
            if set == "train":
                train_set.append((img_arr, label))
            else:
                val_set.append((img_arr, label))

random.shuffle(train_set)
random.shuffle(val_set)

train_X = []
train_Y = []
val_X = []
val_Y = []

for x, y in train_set:
    train_X.append(x)
    train_Y.append(y)

for x, y in val_set:
    val_X.append(x)
    val_Y.append(y)

train_X = np.array(train_X)
train_X = tf.keras.utils.normalize(train_X, axis=1)
train_Y = np.array(train_Y)
val_X = np.array(val_X)
val_X = tf.keras.utils.normalize(val_X, axis=1)
val_Y = np.array(val_Y)

pickle_out = open("hh_train_X.pickle", "wb")
pickle.dump(train_X, pickle_out)
pickle_out.close()

pickle_out = open("hh_train_Y.pickle", "wb")
pickle.dump(train_Y, pickle_out)
pickle_out.close()

pickle_out = open("hh_val_X.pickle", "wb")
pickle.dump(val_X, pickle_out)
pickle_out.close()

pickle_out = open("hh_val_Y.pickle", "wb")
pickle.dump(val_Y, pickle_out)
pickle_out.close()

DENSE MODEL

In [4]:
pickle_in = open("hh_train_X.pickle", "rb")
train_X = pickle.load(pickle_in)
pickle_in = open("hh_train_Y.pickle", "rb")
train_Y = pickle.load(pickle_in)
pickle_in = open("hh_val_X.pickle", "rb")
val_X = pickle.load(pickle_in)
pickle_in = open("hh_val_Y.pickle", "rb")
val_Y = pickle.load(pickle_in)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_X, train_Y, epochs=100)

val_loss, val_acc = model.evaluate(val_X, val_Y)
print(val_loss, val_acc)
model.save('hh_dense.model')

Epoch 1/100
33/33 [==============================] - 1s 4ms/step - loss: 0.6878 - accuracy: 0.5326
Epoch 2/100
33/33 [==============================] - 0s 4ms/step - loss: 0.6789 - accuracy: 0.5901
Epoch 3/100
33/33 [==============================] - 0s 5ms/step - loss: 0.6102 - accuracy: 0.6475
Epoch 4/100
33/33 [==============================] - 0s 5ms/step - loss: 0.5260 - accuracy: 0.7575
Epoch 5/100
33/33 [==============================] - 0s 5ms/step - loss: 0.4783 - accuracy: 0.7838
Epoch 6/100
33/33 [==============================] - 0s 5ms/step - loss: 0.4056 - accuracy: 0.8189
Epoch 7/100
33/33 [==============================] - 0s 5ms/step - loss: 0.3317 - accuracy: 0.8608
Epoch 8/100
33/33 [==============================] - 0s 4ms/step - loss: 0.3318 - accuracy: 0.8530
Epoch 9/100
33/33 [==============================] - 0s 5ms/step - loss: 0.2392 - accuracy: 0.9056
Epoch 10/100
33/33 [==============================] - 0s 5ms/step - loss: 0.2457 - accuracy: 0.8997
Epoch 11/

CONV2D MODEL

In [5]:
pickle_in = open("hh_train_X.pickle", "rb")
train_X = pickle.load(pickle_in)
pickle_in = open("hh_train_Y.pickle", "rb")
train_Y = pickle.load(pickle_in)
pickle_in = open("hh_val_X.pickle", "rb")
val_X = pickle.load(pickle_in)
pickle_in = open("hh_val_Y.pickle", "rb")
val_Y = pickle.load(pickle_in)

NUM_CHANNELS = 1
IMAGE_SIZE = 50
train_X = train_X.reshape(train_X.shape[0], IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)
val_X = val_X.reshape(val_X.shape[0], IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=train_X.shape[1:], activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=train_X.shape[1:], activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_X, train_Y, epochs=25)

val_loss, val_acc = model.evaluate(val_X, val_Y)
print(val_loss, val_acc)
model.save('hh_conv2d.model')

Epoch 1/25
33/33 [==============================] - 4s 97ms/step - loss: 0.6124 - accuracy: 0.7264
Epoch 2/25
33/33 [==============================] - 3s 97ms/step - loss: 0.3941 - accuracy: 0.8354
Epoch 3/25
33/33 [==============================] - 3s 99ms/step - loss: 0.2810 - accuracy: 0.9007
Epoch 4/25
33/33 [==============================] - 3s 101ms/step - loss: 0.2156 - accuracy: 0.9289
Epoch 5/25
33/33 [==============================] - 3s 96ms/step - loss: 0.1836 - accuracy: 0.9387
Epoch 6/25
33/33 [==============================] - 3s 96ms/step - loss: 0.1699 - accuracy: 0.9455
Epoch 7/25
33/33 [==============================] - 3s 96ms/step - loss: 0.1358 - accuracy: 0.9581
Epoch 8/25
33/33 [==============================] - 3s 99ms/step - loss: 0.1271 - accuracy: 0.9552
Epoch 9/25
33/33 [==============================] - 3s 100ms/step - loss: 0.1506 - accuracy: 0.9406
Epoch 10/25
33/33 [==============================] - 3s 104ms/step - loss: 0.0911 - accuracy: 0.9688
Epoch 